# ARAD (Autoencoder Reconstruction Anomaly Detection)

This notebook demonstrates the ARAD algorithm for identifying radioactive sources in gamma-ray time series data using a trained autoencoder.

## Algorithm Overview

The ARAD detector:
1. **Learns background patterns** using a convolutional autoencoder on source-absent training data
2. **Computes reconstruction error** using Chi Squared between input and reconstruction
3. **Detects anomalies** when reconstruction error exceeds threshold
4. **Aggregates alarms** that are close in time

## Key Features
- **Deep learning**: Uses convolutional autoencoder to capture complex spectral patterns
- **Chi2 metric**: More robust than MSE for spectral comparison
- **Unsupervised**: Only needs background data for training
- **Interpretable**: Reconstruction error + saliency maps show what's anomalous

## Dataset

Using the TopCoder Urban Data Challenge dataset (mobile NaI detector).


## Setup


In [ ]:
# Imports
import sys
sys.path.insert(0, '../../gammaflow')
sys.path.insert(0, '..')

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm

# GammaFlow imports
from gammaflow import Spectrum, SpectralTimeSeries, ListMode
from gammaflow.core.spectra import Spectra
from gammaflow.visualization import plot_count_rate_time_series

# Detection algorithm
from src.detectors import ARADDetector

# Configure plotting
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

%matplotlib inline

print("✅ ARAD Detection - Ready!")


## Load Data Helper


In [ ]:
def load_listmode_run(run_id, data_dir='../topcoder', dataset='training'):
    """Load a run and return ListMode object + metadata."""
    data_path = Path(data_dir)
    
    # Load listmode data
    run_file = data_path / dataset / f"{run_id}.csv"
    data = pd.read_csv(run_file, header=None, names=['time_delta_us', 'energy_keV'])
    
    # Convert to seconds
    time_deltas = data['time_delta_us'].values * 1e-6
    energies = data['energy_keV'].values
    
    # Load metadata
    answer_key_file = data_path / 'scorer' / f'answerKey_{dataset}.csv'
    answer_key = pd.read_csv(answer_key_file)
    metadata = answer_key[answer_key['RunID'] == run_id].iloc[0].to_dict()
    
    # Map SourceID to source name
    source_map = {
        0: 'Background',
        1: 'HEU',
        2: 'WGPu',
        3: 'I-131',
        4: 'Co-60',
        5: 'Tc-99m',
        6: 'Tc-99m + HEU'
    }
    metadata['SourceName'] = source_map.get(metadata['SourceID'], f"Unknown({metadata['SourceID']})")
    
    # Create ListMode
    listmode = ListMode(time_deltas, energies)
    
    return listmode, metadata

print("✅ Load function ready")


## Load Trained ARAD Model

We'll load the ARAD model that was trained on background data using the `train_arad.py` script.


In [ ]:
# Load the trained ARAD detector
model_path = '../models/arad_background.pt'

print(f"Loading trained ARAD model from {model_path}...")

detector = ARADDetector(
    latent_dim=8,
    dropout=0.2,
    aggregation_gap=2.0,  # Merge alarms < 2 seconds apart
    verbose=True,
    loss_type='chi2'
)

detector.load(model_path)

print(f"\n✅ Model loaded successfully!")
print(f"   Model config:")
print(f"     - Latent dimension: {detector.latent_dim}")
print(f"     - Number of bins: {detector.n_bins_}")
print(f"     - Device: {detector.device}")
print(f"     - Current threshold: {detector.threshold if detector.threshold is not None else 'Not set'}")


## Set Threshold Based on False Alarm Rate

We'll calibrate the threshold using the **alarms per hour** metric, which is the standard for operational radiation detection systems. ANSI N42.48 typically requires **< 1 alarm/hour** for nuisance alarm rates.


In [ ]:
# Load background time series for threshold calibration
# NOTE: Using just the first background run (same approach as SAD detector)
# This is faster and provides a fair comparison between algorithms.
#
# ⚠️  FOR REAL-WORLD/PRODUCTION USE: You should calibrate on MULTIPLE background
#     runs (e.g., 20+) to get robust statistics and account for environmental
#     variations, detector drift, and different background conditions. Using a
#     single run is acceptable for testing and algorithm comparison, but not
#     for operational deployment.
print("Loading background time series for threshold calibration...")

data_path = Path('../topcoder')
answer_key = pd.read_csv(data_path / 'scorer' / 'answerKey_training.csv')
background_run_ids = answer_key[answer_key['SourceID'] == 0]['RunID'].values

# Load just the first background run (matches SAD's approach)
print(f"Using first background run (ID: {background_run_ids[0]}) for calibration...")
listmode, _ = load_listmode_run(background_run_ids[0])
calibration_data = SpectralTimeSeries.from_list_mode(
    listmode,
    integration_time=3.0,  # 1-second integration
    stride_time=1.0,
    energy_bins=128,
    energy_range=(20, 2900)
)

print(f"\nCalibration dataset: {calibration_data.n_spectra} spectra")
total_time_hours = np.sum(calibration_data.real_times) / 3600.0
print(f"  Total observation time: {total_time_hours:.2f} hours")

# Set threshold
alarms_per_hour = 0.5  # Conservative target (ANSI compliant)

print(f"\nSetting threshold for {alarms_per_hour} alarms per hour...")
threshold = detector.set_threshold_by_far(calibration_data, alarms_per_hour=alarms_per_hour, max_iterations=100)

print(f"\n✅ Threshold set: {threshold:.4f}")


## Test on Run with Source

Now let's test the detector on a run with a radioactive source.


In [ ]:
# Load I-131 run with strongest source
i131_runs = answer_key[answer_key['SourceID'] == 3].copy()
i131_runs = i131_runs.sort_values('Speed/Offset', ascending=False)

# Pick the strongest run
test_run_id = i131_runs.iloc[0]['RunID']
speed_offset = i131_runs.iloc[0]['Speed/Offset']

print(f"Loading I-131 test run {test_run_id}...")
print(f"  Speed/Offset: {speed_offset:.2f} (stronger is better)")
listmode, metadata = load_listmode_run(int(test_run_id))

print(f"\nRun Metadata:")
print(f"  Source: {metadata['SourceName']}")
print(f"  Source Time: {metadata['SourceTime']:.1f} seconds")
print(f"  Speed/Offset: {metadata['Speed/Offset']:.2f}")
print(f"\n{listmode}")

# Convert to time series with 1-second integration
print("\nConverting to SpectralTimeSeries...")
test_time_series = SpectralTimeSeries.from_list_mode(
    listmode,
    integration_time=3.0,
    stride_time=1.0,
    energy_bins=128,
    energy_range=(20, 2900)
)

print(f"\nCreated: {test_time_series}")
print(f"  Number of spectra: {test_time_series.n_spectra}")
print(f"  Time coverage: {test_time_series.timestamps[0]:.1f} to {test_time_series.timestamps[-1]:.1f} s")


## Run ARAD Detection


In [ ]:
print(f"Running ARAD Detection")
print(f"{'='*70}")
print(f"  Latent dimension: {detector.latent_dim}")
print(f"  Threshold: {detector.threshold:.4f}")
print(f"  Device: {detector.device}")
print()

# Process time series
arad_scores, alarms = detector.detect(test_time_series)

print(f"\n{'='*70}")
print(f"Detection Results")
print(f"{'='*70}")
print(f"Number of alarms: {len(alarms)}")

if len(alarms) > 0:
    total_alarm_time = sum(alarm['end_time'] - alarm['start_time'] for alarm in alarms)
    mean_duration = total_alarm_time / len(alarms)
    max_score = max(alarm['peak_score'] for alarm in alarms)
    
    print(f"Total alarm time: {total_alarm_time:.2f} seconds")
    print(f"Mean alarm duration: {mean_duration:.2f} seconds")
    print(f"Peak ARAD score: {max_score:.4f}")
    
    print(f"\nAlarm Events:")
    true_source_time = metadata['SourceTime']
    for i, alarm in enumerate(alarms, 1):
        duration = alarm['end_time'] - alarm['start_time']
        print(f"  {i}. Alarm: t=[{alarm['start_time']:.1f}, {alarm['end_time']:.1f}]s, "
              f"peak={alarm['peak_score']:.4f} at t={alarm['peak_time']:.1f}s, duration={duration:.1f}s")
        
        # Compare to ground truth
        if alarm['start_time'] <= true_source_time <= alarm['end_time']:
            print(f"      ✅ Captured true source (t={true_source_time:.1f}s)")
        else:
            time_diff = min(abs(alarm['start_time'] - true_source_time),
                          abs(alarm['end_time'] - true_source_time))
            print(f"      ⚠️  Offset from true source: {time_diff:.1f}s")
else:
    print("No alarms detected!")


# Visualize Results

In [ ]:
# Plot ARAD scores with alarm overlay
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10), sharex=True)

# Get times and count rates
times = test_time_series.timestamps
count_rates = np.array([
    float(test_time_series[i].counts.sum()) / float(
        test_time_series[i].live_time 
        if (test_time_series[i].live_time is not None and not np.isnan(test_time_series[i].live_time))
        else test_time_series[i].real_time
    )
    for i in range(test_time_series.n_spectra)
])

# Plot 1: Count rate
ax1.step(times, count_rates, where='post', color='black', linewidth=1.5, label='Count rate')
ax1.set_ylabel(r'Count Rate (s$^{-1}$)', fontsize=12)
ax1.set_title(f'ARAD Detection Results - Run {test_run_id} ({metadata["SourceName"]})', 
              fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)

# Highlight alarms
for alarm in alarms:
    ax1.axvspan(alarm['start_time'], alarm['end_time'], alpha=0.3, color='red', label='Alarm')

# Mark true source time
ax1.axvline(metadata['SourceTime'], color='green', linestyle='--', linewidth=2, 
            label=f"True source (t={metadata['SourceTime']:.1f}s)")

# Remove duplicate labels
handles, labels = ax1.get_legend_handles_labels()
by_label = dict(zip(labels, handles))
ax1.legend(by_label.values(), by_label.keys(), fontsize=11)

# Plot 2: ARAD scores
ax2.step(times, arad_scores, where='post', color='steelblue', linewidth=1.5, label='ARAD Score')
ax2.axhline(detector.threshold, color='red', linestyle='--', linewidth=2, 
            label=f'Threshold ({detector.threshold:.4f})')
ax2.set_xlabel('Time (s)', fontsize=12)
ax2.set_ylabel('ARAD Score (CHI 2)', fontsize=12)
ax2.set_title('ARAD Score Time Series', fontsize=13, fontweight='bold')
ax2.grid(True, alpha=0.3)

# Highlight alarms
for alarm in alarms:
    ax2.axvspan(alarm['start_time'], alarm['end_time'], alpha=0.3, color='red')

# Mark true source time
ax2.axvline(metadata['SourceTime'], color='green', linestyle='--', linewidth=2)

ax2.legend(fontsize=11)

plt.tight_layout()
plt.show()

print("\n✅ Detection visualization complete!")


## Summary

The ARAD detector successfully identified the source by learning background patterns and detecting when spectra deviate from the learned representation. The reconstruction error (CHI2) is higher for source-present spectra, enabling detection.
